<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto </font></h1>

## Introduction
In this project, we will explore, segment, and cluster the neighborhoods in the city of Toronto. A Wikipedia page exists that has all the information for Toronto neighborhood data.First, we will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, we can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## Install required libraries

In [2]:
!pip install --upgrade beautifulsoup4
!pip install --upgrade geopy
!pip install --upgrade folium
!pip install --upgrade geocoder

     |████████████████████████████████| 122kB 7.4MB/s eta 0:00:01
  Found existing installation: beautifulsoup4 4.7.1
    Uninstalling beautifulsoup4-4.7.1:
      Successfully uninstalled beautifulsoup4-4.7.1
     |████████████████████████████████| 122kB 9.8MB/s eta 0:00:01
  Found existing installation: geopy 1.18.1
    Uninstalling geopy-1.18.1:
      Successfully uninstalled geopy-1.18.1
     |████████████████████████████████| 102kB 8.9MB/s ta 0:00:011
     |████████████████████████████████| 102kB 7.8MB/s ta 0:00:011


## Import required libraries

In [3]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim     # convert an address into latitude and longitude values
import requests                           # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans 
from bs4 import BeautifulSoup
import xml 
import folium                             # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Load Toronto Neighborhood Data in a dataframe: 
Scrap data from Wikipedia using BeautifulSoup

In [4]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [5]:
Toronto_table = soup.find('table')
postcode = []
borough  = []
neighbourhood = []

#------------metdhod (1)------------------------------------------------
# cell = Toronto_table.find_all('td')
# for i in range(0, len(cell), 3):
#     postcode.append(cell[i].text.strip())    # can replace .text.strip() with .find(text=True) that gives the same results
#     borough .append(cell[i+1].text.strip())
#     neighbourhood.append(cell[i+2].text.strip())
# df = pd.DataFrame([postcode, borough, neighbourhood]).transpose()
# df.columns = ['Postcode', 'Borough', 'Neighbourhood']
# df.head(10)


#------------metdhod (2)------------------------------------------------
# for row in Toronto_table.find_all('tr'):
#     cell = row.find_all('td')
#     if len(data)==3:
#         postcode.append(cell[0].text.strip())    # can replace .text.strip() with .find(text=True) that gives the same results
#         borough .append(cell[1].text.strip())
#         neighbourhood.append(cell[2].text.strip())    
        
# df = pd.DataFrame([postcode, borough, neighbourhood]).transpose()
# df.columns = ['Postcode', 'Borough', 'Neighbourhood']
# df.head(10)

#------------metdhod (3)------------------------------------------------
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns = column_names)
# Search all the postcode, borough, neighborhood 
for tr in Toronto_table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        neighborhoods.loc[len(neighborhoods)] = row_data
print('Data scrapped!')

Data scrapped!


Quickly examine the resulting dataframe.

In [6]:
neighborhoods.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 3 columns):
Postalcode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


In [7]:
neighborhoods.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Check the number of boroughs and neighborhoods.

In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 180 neighborhoods.


## Data Cleaning

In [9]:
neighborhoods['Borough'].replace('Not assigned', np.nan, inplace=True)
neighborhoods.dropna(subset=['Borough'], inplace=True)
neighborhoods['Neighborhood'].replace('', neighborhoods['Borough'], inplace=True)

neighborhoods.info()

# df=df[df['Borough']!='Not assigned']
# df[df['Neighborhood']==" "]=df['Borough']
# df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 3 columns):
Postalcode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 8.2+ KB


In [10]:
neighborhoods_grouped = neighborhoods.groupby(['Postalcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
neighborhoods_grouped.columns = ['Postalcode', 'Borough', 'Neighborhood']
neighborhoods_grouped.info()

# neighborhoods_grouped=neighborhoods.groupby(['Postalcode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index(drop=False)
# neighborhoods_grouped.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
# neighborhoods_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postalcode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [11]:
neighborhoods_grouped['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)

neighborhoods_grouped

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
neighborhoods_grouped.shape

(103, 3)

## Geospatial Data of Neighborhoods

In [13]:
# import geocoder
# def get_geocoder(postal_code_from_df):
#      # initialize your variable to None
#      lat_lng_coords = None
#      # loop until you get the coordinates
#      while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code_from_df))
#        lat_lng_coords = g.latlng
#      latitude = lat_lng_coords[0]
#      longitude = lat_lng_coords[1]
#      return latitude,longitude
# df_position = df_grouped
# for i in range(0,len(df_position)):
#     #df_position['Latitude'][i],df_position['Longitude'][i]=get_geocoder(df_grouped.iloc[i]['Postalcode']
#    df_position['Latitude'][i],df_position['Longitude'][i]=get_geocoder(df_position['Postalcode'].values)   # using np.vectorize to speed up.                                                                 
#    #df_position['Latitude'], df_position['Longitude'] = zip(*df_position['Postalcode'].apply(get_geocoder))
   


In [14]:
# csv file that has the geographical coordinates of each postal code
neighborhoods_geospatial = pd.read_csv('http://cocl.us/Geospatial_data')
neighborhoods_geospatial.columns = ['Postalcode', 'Latitude', 'Longitude']

In [15]:
neighborhoods_position = pd.merge(neighborhoods_grouped, neighborhoods_geospatial, on=['Postalcode'], how='inner')
toronto_data = neighborhoods_position[['Borough', 'Neighborhood', 'Postalcode', 'Latitude', 'Longitude']].copy()
toronto_data.head()

,Borough,Neighborhood,Postalcode,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


Let's get the geographical coordinates of Toronto.

In [16]:
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto  with neighborhoods superimposed on top.

In [17]:
# creating map of Toronto using latitude, longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [18]:
# @hidden_cell
CLIENT_ID = '2KSOWMNUTD3UQMGXDA3F0QRZHEWBCCA03X4IOJY0PPT21CYV' # your Foursquare ID
CLIENT_SECRET = 'STUNTIEU53FRY0NBEARBDSZAQVH0KCXX1SQJQWMZMKMI1UHZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2KSOWMNUTD3UQMGXDA3F0QRZHEWBCCA03X4IOJY0PPT21CYV
CLIENT_SECRET:STUNTIEU53FRY0NBEARBDSZAQVH0KCXX1SQJQWMZMKMI1UHZ


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [19]:
toronto_data.loc[0, 'Neighborhood'] #exploring first neighbourhood

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values.

In [20]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude']   # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto_data.loc[0, 'Neighborhood']   # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Malvern, Rouge within a radius of 1000 meters.

First, let's create the GET request URL. Name your URL **url**.

In [46]:
LIMIT = 100
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2KSOWMNUTD3UQMGXDA3F0QRZHEWBCCA03X4IOJY0PPT21CYV&client_secret=STUNTIEU53FRY0NBEARBDSZAQVH0KCXX1SQJQWMZMKMI1UHZ&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=100'

Send the GET request and examine the resutls

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ecdc7919c6f59001b872ab5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b6718c2f964a5203f3a2be3',
       'name': "Harvey's",
       'location': {'address': '853 Milner Ave',
        'crossStreet': 'at Morningside Ave',
        'lat': 43.80002047583462,
        'lng': -79.19830662127322,
        'labeledLatLngs': [{'label':

extract the category of the venue

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [49]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Harvey's,Restaurant,43.800020,-79.198307
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
3,RBC Royal Bank,Bank,43.798782,-79.197090
4,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777


In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [52]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'],
                                   radius=500)

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

#### Let's check the size of the resulting dataframe

In [54]:
print(toronto_venues.shape)
toronto_venues.head()

(2126, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


Let's check how many venues were returned for each neighborhood

In [55]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18


#### Let's find out how many unique categories can be curated from all the returned venues

In [56]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


## 3. Analyze Each Neighborhood

In [57]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [58]:
toronto_onehot.shape

(2126, 275)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [59]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

#### Let's confirm the new size

In [60]:
toronto_grouped.shape

(94, 275)

#### Let's print each neighborhood along with the top 5 most common venues

In [61]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1               Skating Rink  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4                Yoga Studio  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1  Skating Rink  0.11
2   Coffee Shop  0.11
3           Pub  0.11
4  Dance Studio  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.09
1            Bank  0.09
2  Ice Cream Shop  0.04
3     Supermarket  0.04
4      Restaurant  0.04


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0       Italian Restaurant  0.08
1           Sandwich Place  0.08
2              Coffee Shop  0.08
3               Restaurant  0

                             venue  freq
0                      Pizza Place   1.0
1                    Metro Station   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0


----Humberlea, Emery----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Humewood-Cedarvale----
           venue  freq
0          Field  0.25
1   Tennis Court  0.25
2   Hockey Arena  0.25
3          Trail  0.25
4  Metro Station  0.00


----India Bazaar, The Beaches West----
                  venue  freq
0  Fast Food Restaurant  0.11
1             Pet Store  0.05
2           Coffee Shop  0.05
3         Movie Theater  0.05
4        Sandwich Place  0.05


----Kennedy Park, Ionview, East Birchmount Park----
                venue  freq
0    Depart

                 venue  freq
0                 Café  0.06
1          Coffee Shop  0.06
2         Cocktail Bar  0.05
3            Gastropub  0.04
4  American Restaurant  0.04


----St. James Town, Cabbagetown----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.07
2                Park  0.05
3                 Pub  0.05
4  Italian Restaurant  0.05


----Steeles West, L'Amoreaux West----
                  venue  freq
0  Fast Food Restaurant  0.12
1    Chinese Restaurant  0.12
2              Pharmacy  0.06
3           Coffee Shop  0.06
4        Discount Store  0.06


----Stn A PO Boxes----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.04
2           Restaurant  0.03
3  Japanese Restaurant  0.03
4             Beer Bar  0.03


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3               Bakery  0.05
4  American Restaurant  0.05



#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Dance Studio,Sandwich Place,Coffee Shop,Gastropub,Gas Station
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Pizza Place,Ice Cream Shop,Supermarket,Sushi Restaurant,Middle Eastern Restaurant,Deli / Bodega,Restaurant
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Italian Restaurant,Liquor Store,Pharmacy,Butcher,Pub,Café,Sushi Restaurant


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [64]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [65]:
# add clustering labels
if ('Cluster Labels' in neighborhoods_venues_sorted.columns):                    # not in 
    neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)
toronto_merged# check the last columns!

,Borough,Neighborhood,Postalcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353,1.0,Print Shop,Fast Food Restaurant,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497,1.0,Bar,History Museum,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,1.0,Electronics Store,Rental Car Location,Breakfast Spot,Mexican Restaurant,Intersection,Medical Center,Bank,Diner,Dessert Shop,Dim Sum Restaurant
3,Scarborough,Woburn,M1G,43.770992,-79.216917,1.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,1.0,Lounge,Caribbean Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Thai Restaurant,Gas Station,Hakka Restaurant,Bakery,Donut Shop
5,Scarborough,Scarborough Village,M1J,43.744734,-79.239476,1.0,Spa,Playground,Cosmetics Shop,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
6,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",M1K,43.727929,-79.262029,1.0,Convenience Store,Department Store,Coffee Shop,Discount Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dog Run
7,Scarborough,"Golden Mile, Clairlea, Oakridge",M1L,43.711112,-79.284577,1.0,Bakery,Bus Line,Soccer Field,Bus Station,Park,Ice Cream Shop,Comic Shop,Colombian Restaurant,College Rec Center,Eastern European Restaurant
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",M1M,43.716316,-79.239476,1.0,Motel,American Restaurant,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
9,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848,1.0,College Stadium,Skating Rink,Café,General Entertainment,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


Finally, let's visualize the resulting clusters

In [66]:
import folium
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster =int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Victoria Village,0.0,Hockey Arena,French Restaurant,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,Women's Store,Deli / Bodega,Department Store,Dessert Shop
72,Glencairn,0.0,Pizza Place,Japanese Restaurant,Park,Sushi Restaurant,Pub,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
89,"Alderwood, Long Branch",0.0,Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Dance Studio,Sandwich Place,Coffee Shop,Gastropub,Gas Station
96,Humber Summit,0.0,Pizza Place,Women's Store,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
99,Westmount,0.0,Pizza Place,Discount Store,Middle Eastern Restaurant,Coffee Shop,Intersection,Sandwich Place,Chinese Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant
100,"Kingsview Village, St. Phillips, Martin Grove ...",0.0,Mobile Phone Shop,Bus Line,Pizza Place,Sandwich Place,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store


#### Cluster 2

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",1.0,Print Shop,Fast Food Restaurant,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"Rouge Hill, Port Union, Highland Creek",1.0,Bar,History Museum,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Guildwood, Morningside, West Hill",1.0,Electronics Store,Rental Car Location,Breakfast Spot,Mexican Restaurant,Intersection,Medical Center,Bank,Diner,Dessert Shop,Dim Sum Restaurant
3,Woburn,1.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,Cedarbrae,1.0,Lounge,Caribbean Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Thai Restaurant,Gas Station,Hakka Restaurant,Bakery,Donut Shop
5,Scarborough Village,1.0,Spa,Playground,Cosmetics Shop,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
6,"Kennedy Park, Ionview, East Birchmount Park",1.0,Convenience Store,Department Store,Coffee Shop,Discount Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dog Run
7,"Golden Mile, Clairlea, Oakridge",1.0,Bakery,Bus Line,Soccer Field,Bus Station,Park,Ice Cream Shop,Comic Shop,Colombian Restaurant,College Rec Center,Eastern European Restaurant
8,"Cliffside, Cliffcrest, Scarborough Village West",1.0,Motel,American Restaurant,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
9,"Birch Cliff, Cliffside West",1.0,College Stadium,Skating Rink,Café,General Entertainment,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


#### Cluster 3

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,Weston,2.0,Convenience Store,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Deli / Bodega


#### Cluster 4

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Milliken, Agincourt North, Steeles East, L'Amo...",3.0,Park,Playground,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
23,York Mills West,3.0,Convenience Store,Park,Bank,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
25,Parkwoods,3.0,Park,Food & Drink Shop,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
40,"East Toronto, Broadview North (Old East York)",3.0,Park,Convenience Store,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
44,Lawrence Park,3.0,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dance Studio
50,Rosedale,3.0,Park,Trail,Playground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
64,"Forest Hill North & West, Forest Hill Road Park",3.0,Park,Sushi Restaurant,Jewelry Store,Trail,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
74,Caledonia-Fairbanks,3.0,Park,Pool,Women's Store,Golf Course,Cupcake Shop,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 5

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,"Humberlea, Emery",4.0,Baseball Field,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Deli / Bodega
